# MESH vector-based workflow: make folder structure
This script is adapted from the SUMMA workflow https://github.com/CH-Earth/CWARHM

Makes the initial folder structure for a given control file. All other files in the workflow will look for the file `control_active.txt` during their execution. This script:

1. Copies the specified control file into `control_active.txt`;
2. Prepares a folder structure using the settings in `control_active.txt`.
3. Creates a copy of itself to be stored in the new folder structure.

The destination folders are referred to as "domain folders".

In [37]:
# Specify the control file to use
sourceFile  = 'control_BowAtBanff.txt'

Do not change below this line.

In [38]:
# Modules
import os
from pathlib import Path
from shutil import copyfile
from datetime import datetime

#### Copy the control file into `control_active.txt`

In [39]:
# Easy access to control file folder
controlFolder = Path('../0_control_files')

In [40]:
# Store the name of the 'active' file in a variable
controlFile = 'control_active.txt'

In [41]:
copyfile( controlFolder/sourceFile, controlFolder/controlFile );

#### Create the main domain folder

In [42]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
            
            # ... find the line with the requested setting
            if setting in line and not line.startswith('#'):
                break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
       
    # Return this value    
    return substring

In [43]:
# Find the path where the domain folders need to go
# Immediately store as a 'Path' to avoid issues with '/' and '\' on different operating systems
rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )

In [44]:
# Find the domain name
domainName = read_from_control(controlFolder/controlFile,'domain_name')

In [45]:
# Get the domain folder
domainFolder = 'domain_' + domainName


#### Make the shapefile folders

In [46]:
# Find the catchment shapefile folder in 'control_active'
networkShapeFolder = read_from_control(controlFolder/controlFile,'river_network_shp_path')
riverBasinFolder =  read_from_control(controlFolder/controlFile,'river_basin_shp_path')

In [47]:
# Specify the default paths if required
if riverBasinFolder == 'default':
    riverBasinFolder = 'shapefiles/catchment'
if networkShapeFolder == 'default':
    networkShapeFolder = 'shapefiles/river_network'

In [48]:
# Try to make the shapefile folders; does nothing if the folder already exists
Path( rootPath / networkShapeFolder).mkdir(parents=True, exist_ok=True)
Path( rootPath / riverBasinFolder ).mkdir(parents=True, exist_ok=True)

In [49]:
rootPath / networkShapeFolder

WindowsPath('C:/Users/5600x/Desktop/GWF/MESH-Scripts/Model_Workflow/shapefiles/river_network')

#### Make the simulation folders

In [50]:
simulationFolder = read_from_control(controlFolder/controlFile,'simulation_path')

In [51]:
if simulationFolder == 'default':
    simulationFolder = 'simulations'

In [52]:
Path( rootPath / 'vector_based_workflow/workflow_data' / domainFolder / simulationFolder ).mkdir(parents=True, exist_ok=True)

#### Make the visualization folders

In [53]:
visualizationFolder = read_from_control(controlFolder/controlFile,'visualization_folder')

In [54]:
if visualizationFolder == 'default':
    visualizationFolder = 'visualizations'

In [55]:
Path( rootPath / 'vector_based_workflow/workflow_data' / domainFolder / visualizationFolder ).mkdir(parents=True, exist_ok=True)

#### Make the forcing folder

In [56]:
forcingFolder = read_from_control(controlFolder/controlFile,'source_nc_path')

In [57]:
if forcingFolder == 'default':
    forcingFolder = 'forcing'

In [58]:
Path( rootPath / forcingFolder ).mkdir(parents=True, exist_ok=True)

#### Make the installs folder

In [59]:
installsFolder = read_from_control(controlFolder/controlFile,'install_path_MESH')

In [60]:
if installsFolder == 'default':
    installsFolder = 'installs'

In [61]:
Path( rootPath / installsFolder ).mkdir(parents=True, exist_ok=True)

#### Make the parameter folders

In [62]:
drainageDatabaseFolder  = read_from_control(controlFolder/controlFile,'DDB_output_dir')
networkTopologyFolder   = read_from_control(controlFolder/controlFile,'settings_routing_path')
zonalStatisticsFolder   = read_from_control(controlFolder/controlFile,'input_lc_zh_path')

In [63]:
if drainageDatabaseFolder   == 'default':
    drainageDatabaseFolder = 'drainagedatabase'
if networkTopologyFolder    == 'default':
    networkTopologyFolder   = 'topology'
if zonalStatisticsFolder    == 'default':
    zonalStatisticsFolder   = 'zonalhist'

In [64]:
Path( rootPath / 'vector_based_workflow/workflow_data' / domainFolder / drainageDatabaseFolder  ).mkdir(parents=True, exist_ok=True)
Path( rootPath / 'vector_based_workflow/workflow_data' / domainFolder / networkTopologyFolder   ).mkdir(parents=True, exist_ok=True)
Path( rootPath / 'vector_based_workflow/workflow_data' / domainFolder / zonalStatisticsFolder   ).mkdir(parents=True, exist_ok=True)

#### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.

In [65]:
# Create a log folder
logFolder = '_workflow_log'
Path( rootPath / 'vector_based_workflow/workflow_data' / domainFolder / logFolder ).mkdir(parents=True, exist_ok=True)

In [66]:
# Create a log folder
logFolder = '_workflow_log'
Path( rootPath / 'vector_based_workflow/workflow_data' / domainFolder / logFolder ).mkdir(parents=True, exist_ok=True)

In [67]:
# Copy this script
thisFile = 'make_folder_structure.ipynb'
copyfile(thisFile, rootPath / 'vector_based_workflow/workflow_data' / domainFolder / logFolder / thisFile);

In [68]:
# Get current date and time
now = datetime.now()

In [69]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + '_log.txt'
with open(rootPath / 'vector_based_workflow/workflow_data' / domainFolder / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Generated folder structure using ' + sourceFile]
    for txt in lines:
        file.write(txt)    